In [15]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import numpy as np
import pandas as pd

# Gráficos
# ------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns


#  Modelado y evaluación
# -----------------------------------------------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import pickle
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV

In [16]:
df = pd.read_pickle("../data/bikes_casuales_estand_encod.pkl")
df.head()

,año,mes,festivo,dia,meteo,temperatura,humedad,velo_viento,usu_casuales
0,0,0,1,3,1,-0.486274,0.858854,-0.208909,331
1,0,0,0,2,1,-0.425690,0.331548,0.687550,131
2,0,0,0,1,2,-0.949970,-0.912000,0.685210,120
3,0,0,0,1,2,-0.938563,-0.176088,-0.210435,108
4,0,0,0,0,2,-0.853992,-0.913519,0.060294,82


In [17]:
X = df.drop("usu_casuales", axis = 1)
y = df["usu_casuales"]

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 15)

arbol = DecisionTreeRegressor(random_state =15)

arbol.fit(x_train, y_train)

DecisionTreeRegressor(random_state=15)

In [18]:
with open ("../data/dt_casuales.pkl", "wb") as f:
    pickle.dump(arbol, f)

In [19]:
def metricas(y_test, y_train, y_test_pred, y_train_pred, tipo_modelo):
    
    
    resultados = {'MAE': [mean_absolute_error(y_test, y_test_pred), mean_absolute_error(y_train, y_train_pred)],
                'MSE': [mean_squared_error(y_test, y_test_pred), mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(mean_squared_error(y_test, y_test_pred)), np.sqrt(mean_squared_error(y_train, y_train_pred))],
                'R2':  [r2_score(y_test, y_test_pred), r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    df["modelo"] = tipo_modelo
    return df

In [20]:
y_pred_test_dt = arbol.predict(x_test)
y_pred_train_dt = arbol.predict(x_train)

In [21]:
max_features = np.sqrt(len(x_train.columns))
max_features

2.8284271247461903

In [22]:
print(arbol.tree_.max_depth)

24


In [23]:
param = {"max_depth": [2, 3, 4],
         "max_features": [2, 3],
         "min_samples_split": [5, 7],
         "min_samples_leaf": [5, 7]} 

In [24]:
gs = GridSearchCV(estimator=DecisionTreeRegressor(), 
                  param_grid= param,
                  cv=10,
                  verbose=0,
                  return_train_score = True,
                  scoring="neg_mean_squared_error")

In [25]:
gs.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [2, 3, 4], 'max_features': [2, 3],
                         'min_samples_leaf': [5, 7],
                         'min_samples_split': [5, 7]},
             return_train_score=True, scoring='neg_mean_squared_error')

In [26]:
with open ("../data/dt_casuales_mejor_modelo.pkl", "wb") as f:
    pickle.dump(gs, f)

In [27]:
mejor_modelo = gs.best_estimator_
mejor_modelo

DecisionTreeRegressor(max_depth=4, max_features=2, min_samples_leaf=5,
                      min_samples_split=5)

In [28]:
dt_results = metricas(y_test, y_train, y_pred_test_dt, y_pred_train_dt, "DT_registrados_estad_encode")
dt_results

,MAE,MSE,RMSE,R2,set,modelo
0,332.780822,250326.986301,500.326879,0.5385,test,DT_registrados_estad_encode
1,0.000000,0.000000,0.000000,1.0000,train,DT_registrados_estad_encode
